In [1]:
import os
import numpy as np
import cv2
import glob
import csv
from matplotlib import pyplot as plt

In [2]:
import numpy as np

def pHash(img_path,l=32,w=32):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (l, w))  
    img=img.astype(np.float32)

    img=cv2.dct(img)
    img=img[0:8,0:8]
    sum=0.
    hash=''


    for i in range(8):
        for j in range(8):
            sum+=img[i,j]
    avg=sum/64


    for i in range(8):
        for j in range(8):
            if img[i,j]>avg:
                hash=hash+'1'
            else:
                hash=hash+'0'
    return hash

def aHash(img_path,l=8,w=8):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (l, w))  
    sum=0
    hash=''
    for i in range(8):
        for j in range(8):
            sum+=img[i,j]
    avg=sum/64
    for i in range(8):
        for j in range(8):
            if  img[i,j]>avg:
                hash=hash+'1'
            else:
                hash=hash+'0'            
    return hash


def dHash(img_path,l=8,w=8):
    img = cv2.imread(img_path)
    img=cv2.resize(img,(9,8),interpolation=cv2.INTER_CUBIC)
    
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    hash=''
    
    for i in range(8):
        for j in range(8):
            if img[i,j]>img[i,j+1]:
                hash=hash+'1'
            else:
                hash=hash+'0'
    return hash

In [3]:
def Hamming_distance(hash1,hash2):
    count = 0
    assert len(hash1)==len(hash2)
    for i in range(len(hash1)):
        if hash1[i] != hash2[i]:
            count += 1
    return (1 - count * 1.0 / 64)

In [4]:
def getMathNum(matches,ratio):
    matchesMask = [[0,0] for i in range(len(matches))]
    matchNum = 0
    for i,(m,n) in enumerate(matches):
        if m.distance < ratio *n.distance: 
            matchesMask[i] = [1,0]
            matchNum += 1
    matchRatio = matchNum*100/len(matches)
    return matchRatio

In [5]:
def siftMatch(img1, img2,ratio,pointnum):
    if pointnum == 0:
        sift = cv2.xfeatures2d.SIFT_create()
    elif pointnum > 0:
        sift = cv2.xfeatures2d.SIFT_create(pointnum)

    
    img1 = cv2.imread(img1, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(img2, cv2.IMREAD_GRAYSCALE)
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)
    
    if kp2 == [] or len(kp2)<5:
        return 0
    
    FLANN_INDEX_KDTREE = 0
    indexParams = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    searchParams = dict(checks=50)
    flann = cv2.FlannBasedMatcher(indexParams, searchParams)
    

    matches = flann.knnMatch(des1, des2, k=2)
    matchRatio = getMathNum(matches, ratio)
#     drawlines(img1,img2,kp1,kp2,matches)
    
    return matchRatio
 
    
def orbMatch(img1, img2,ratio,pointnum):
    
    if pointnum == 0:
        orb = cv2.ORB_create()
    elif pointnum > 0:
        orb = cv2.ORB_create(pointnum)    
    img1 = cv2.imread(img1, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(img2, cv2.IMREAD_GRAYSCALE)
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)
    
    if kp2 == [] or len(kp2)<5:
        return 0
    


    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)
    
    matchRatio = getMathNum(matches, ratio)
#     drawlines(img1,img2,kp1,kp2,matches)
    
    return matchRatio

In [6]:
def siftDraw(img1, img2,ratio,pointnum):
    if pointnum == 0:
        sift = cv2.xfeatures2d.SIFT_create()
    elif pointnum > 0:
        sift = cv2.xfeatures2d.SIFT_create(pointnum)

    
    img1 = cv2.imread(img1, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(img2, cv2.IMREAD_GRAYSCALE)
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)
    
    if kp2 == [] or len(kp2)<5:
        return 0
    
    FLANN_INDEX_KDTREE = 0
    indexParams = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    searchParams = dict(checks=50)
    flann = cv2.FlannBasedMatcher(indexParams, searchParams)

    matches = flann.knnMatch(des1, des2, k=2)
    matchRatio = getMathNum(matches, ratio)
    drawlines(img1,img2,kp1,kp2,matches)
    
    return matchRatio
 
    
def orbDraw(img1, img2,ratio,pointnum):
    
    if pointnum == 0:
        orb = cv2.ORB_create()
    elif pointnum > 0:
        orb = cv2.ORB_create(pointnum)    
    img1 = cv2.imread(img1, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(img2, cv2.IMREAD_GRAYSCALE)
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)
    
    if kp2 == [] or len(kp2)<5:
        return 0
    


    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)
    
    matchRatio = getMathNum(matches, ratio)
    drawlines(img1,img2,kp1,kp2,matches)
    
    return matchRatio

In [7]:
def drawlines(img1,img2,kp1,kp2,matches):
    threshold = 0.75
    matchesMask = [[0,0] for i in range(len(matches))]
    for i,(m,n) in enumerate(matches):
        if m.distance < threshold *n.distance: 
            matchesMask[i] = [1,0]
    drawPrams = dict(matchColor=(0, 0, 255),singlePointColor=(255, 0, 0),matchesMask=matchesMask,flags=0)

    match_img = cv2.drawMatchesKnn(img1, kp1, img2, kp2, matches, None, **drawPrams)

    plt.figure(figsize=(15,15))
    plt.imshow(match_img) 
    plt.title("Feature matching " + ' keypoint:'+str(len(kp1)))
    plt.axis('off')
    plt.show()

In [8]:
import pandas as pd
from tqdm import tqdm
import re

In [9]:
def findBestImage(matchtype,hashtype,ratio,pointnum,distance,test_image_name):
    image_test = cv2.imread(os.path.join(test_images_rootpath,test_image_name),cv2.IMREAD_GRAYSCALE)
    match_p = 0
    dic_dist = {}

    if hashtype == 'phash' :
        hash1 = pHash(test_images_rootpath+test_image_name)
    elif hashtype == 'dhash':
        hash1 = dHash(test_images_rootpath+test_image_name)
    elif hashtype == 'ahash':
        hash1 = aHash(test_images_rootpath+test_image_name)
    for img in tqdm(train_images):
        if re.match(r'(.*?jpg)', img) is not None:
            if hashtype == 'phash' :
                hash2 = pHash(train_images_rootpath+img)
                dist = Hamming_distance(hash1,hash2)
            elif hashtype == 'dhash':
                hash2 = dHash(train_images_rootpath+img)
                dist = Hamming_distance(hash1,hash2)

            elif hashtype == 'ahash':
                hash2 = aHash(train_images_rootpath+img)
                dist = Hamming_distance(hash1,hash2)
                
            if dist > distance:
                if matchtype == 'sift':
                    match1 = siftMatch(test_images_rootpath+test_image_name, train_images_rootpath+img,ratio,pointnum)
                    if match1 > match_p:
                        match_p = match1
                        best_match = train_images_rootpath+img
                        best_image = img
                            
                elif matchtype == 'orb':
                    match1 = orbMatch(test_images_rootpath+test_image_name, train_images_rootpath+img,ratio,pointnum)
                    if match1 > match_p:
                        match_p = match1
                        best_match = train_images_rootpath+img
                        best_image = img
                            
    img2 = cv2.imread(best_match, cv2.IMREAD_GRAYSCALE) 
    
    if matchtype == 'sift':
        siftDraw(test_images_rootpath+test_image_name, best_match,ratio,pointnum)
    elif matchtype == 'orb':
        orbDraw(test_images_rootpath+test_image_name, best_match,ratio,pointnum)

    plt.subplot(1,2,1)
    plt.title(test_images_rootpath+test_image_name+', '+best_match+', '+str(match_p))
    plt.imshow(image_test, cmap='gray') 
    plt.axis('off')

    plt.subplot(1,2,2)
    plt.imshow(img2, cmap='gray') 
    plt.axis('off')
    plt.show()

In [ ]:
def findBestAndExport(matchtype,hashtype,ratio,pointnum):
    rows=[]
    headers = ['id','x','y']
    for image in tqdm(os.listdir(test_images_rootpath)):
        image_test = cv2.imread(test_images_rootpath+image,cv2.IMREAD_GRAYSCALE)
        match_p = 0
        dic_dist = {}
        if hashtype == 'phash' :
            hash1 = pHash(test_images_rootpath+image)
        elif hashtype == 'dhash':
            hash1 = dHash(test_images_rootpath+image)
        elif hashtype == 'ahash':
            hash1 = aHash(test_images_rootpath+image)
        for img in train_images:
            if re.match(r'(.*?jpg)', img) is not None:
                if hashtype == 'phash' :
                    hash2 = pHash(train_images_rootpath+img)
                elif hashtype == 'dhash':
                    hash2 = dHash(train_images_rootpath+img)
                elif hashtype == 'ahash':
                    hash2 = aHash(train_images_rootpath+img)
                dist = Hamming_distance(hash1,hash2)


                if dist > 0.7:
                    if matchtype == 'sift':
                        match1 = siftMatch(test_images_rootpath+image, train_images_rootpath+img,ratio,pointnum)
                        if match1 > match_p:
                            match_p = match1
                            best_match = train_images_rootpath+img
                            best_image = img
                    elif matchtype == 'orb':
                        match1 = orbMatch(test_images_rootpath+image, train_images_rootpath+img,ratio,pointnum)
                        if match1 > match_p:
                            match_p = match1
                            best_match = train_images_rootpath+img
                            best_image = img
        img2 = cv2.imread(best_match, cv2.IMREAD_GRAYSCALE) 
        
        
            
        plt.subplot(1,2,1)
        plt.title(test_images_rootpath+image+', '+best_match+', '+str(match_p))
        plt.imshow(image_test, cmap='gray') 
        plt.axis('off')

        plt.subplot(1,2,2)
        plt.imshow(img2, cmap='gray') 
        plt.axis('off')
        plt.show()
        
        try:
            train_image_name = best_image[:-4]
        except:
            er = {'id':image,'x':0,'y':0}
            rows.append(er)
            continue
        train_image_data = train_data.loc[train_image_name]
        test_image_data = test_data.loc[image[:-4]]
        test_image_data['x'] = train_image_data[0]
        test_image_data['y'] = train_image_data[1]
#             print('test' ,image, 'train' ,train_image_name)
        test_map = {'id':image,'x':test_image_data['x'],'y':test_image_data['y']}
        rows.append(test_map)
    with open('./new.csv','w',newline='')as f:
        f_csv = csv.DictWriter(f,headers)
        f_csv.writeheader()
        f_csv.writerows(rows)

In [ ]:
test_images_rootpath = './COMP90086_2021_Project_test/test/'
train_images_rootpath = './COMP90086_2021_Project_train/train/'
train_images = os.listdir(train_images_rootpath)


train_data = pd.read_csv("./COMP90086_2021_Project_train/train.csv",index_col ="id")
test_data = pd.read_csv("./COMP90086_2021_Project_test/imagenames.csv",index_col ="id")

In [ ]:
import math
W=680
H=490
FOVh=73.7
FOVv=53.1

fx=W/(2*math.tan(math.radians(FOVh)/2))
fy=H/(2*math.tan(math.radians(FOVv)/2))
cx=W/2
cy=H/2

CameraMAT=np.array([[ fx, 0, cx ],
       [ 0, fy, cy ],
       [ 0, 0, 1 ]])
# cv2.findEssentialMat(pts1,pts2,(fx+fy)/2,(cx,cy))
print(fx,fy,cx,cy,CameraMAT,K)

In [ ]:
E = np.dot(np.dot(CameraMAT.T,F),CameraMAT)

In [ ]:
R=np.array(np.zeros((3,3)))
t=np.array(np.zeros((3,3)))
focal_length = (fx+fy)/2
principal_point = (cx,cy)
number, R, t, mask  =cv2.recoverPose( E, pts1, pts2, R, t, focal_length, principal_point, mask=mask[:, None].astype(np.uint8))